In [74]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import hockey_scraper
import pickle
import random



Create function to format each NST csv coming in

In [75]:
def process_df(file_path):
    df=pd.read_csv(file_path)
    df.reset_index(drop=True,inplace=True)
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1, inplace=True)
    return df


Load in all 9 NST dataframes

In [92]:
NHL_5v5_2122=process_df("NHL_5v5_2122.csv")
NHL_5v5_2223=process_df("NHL_5v5_2223.csv")
NHL_5v5_2324=process_df("NHL_5v5_2324.csv")

NHL_PP_2122=process_df("NHL_PP_2122.csv")
NHL_PP_2223=process_df("NHL_PP_2223.csv")
NHL_PP_2324=process_df("NHL_PP_2324.csv")

NHL_PK_2122=process_df("NHL_PK_2122.csv")
NHL_PK_2223=process_df("NHL_PK_2223.csv")
NHL_PK_2324=process_df("NHL_PK_2324.csv")

In [77]:
# Set option to display all columns
pd.set_option('display.max_columns', None)


For some reason in the 21-22 season, TOR and TBL only have 81 games playes instead of 82. Toronto needs another loss and tampa another win. TBL should have 51 wins (110 points) not 50 wins and 108 points. TOR should have 21 losses not 20. 

In [96]:
filtered_df = NHL_5v5_2122[NHL_5v5_2122['Team'].isin(['Toronto Maple Leafs', 'Tampa Bay Lightning'])]
filtered_df

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCSF,SCSA,SCSF%,SCGF,SCGA,SCGF%,SCSH%,SCSV%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO
1,Tampa Bay Lightning,82,3870.4667,51,23,8,49,110,0.671,3550,3426,50.89,2736,2616,51.12,1929,1894,50.46,183,148,55.29,168.55,153.06,52.41,1928,1797,51.76,1119,1063,51.28,156,129,54.74,13.94,87.86,789,679,53.75,555,505,52.36,99,92,51.83,17.84,81.78,1139,1118,50.47,564,558,50.27,57,37,60.64,10.11,93.37,1771,1746,50.36,752,758,49.80,26,19,57.78,3.46,97.49,9.49,92.19,1.017
5,Toronto Maple Leafs,82,3979.4000,54,21,7,51,115,0.701,4074,3533,53.56,3070,2711,53.10,2232,1993,52.83,208,182,53.33,190.58,154.43,55.24,2239,1762,55.96,1337,1070,55.55,182,160,53.22,13.61,85.05,880,693,55.94,637,504,55.83,112,107,51.14,17.58,78.77,1359,1069,55.97,700,566,55.29,70,53,56.91,10.00,90.64,1680,1570,51.69,818,814,50.12,25,22,53.19,3.06,97.30,9.32,90.87,1.002


In [97]:
NHL_5v5_2122.head()

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCSF,SCSA,SCSF%,SCGF,SCGA,SCGF%,SCSH%,SCSV%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO
0,Pittsburgh Penguins,82,4030.0000,46,25,11,43,103,0.628,3905,3598,52.05,3031,2697,52.92,2257,2047,52.44,180,154,53.89,178.82,156.01,53.41,1968,1720,53.36,1247,1085,53.47,157,137,53.40,12.59,87.37,806,683,54.13,645,540,54.43,108,98,52.43,16.74,81.85,1162,1037,52.84,602,545,52.48,49,39,55.68,8.14,92.84,1742,1691,50.74,909,863,51.30,23,17,57.50,2.53,98.03,7.98,92.48,1.005
1,Tampa Bay Lightning,82,3870.4667,51,23,8,49,110,0.671,3550,3426,50.89,2736,2616,51.12,1929,1894,50.46,183,148,55.29,168.55,153.06,52.41,1928,1797,51.76,1119,1063,51.28,156,129,54.74,13.94,87.86,789,679,53.75,555,505,52.36,99,92,51.83,17.84,81.78,1139,1118,50.47,564,558,50.27,57,37,60.64,10.11,93.37,1771,1746,50.36,752,758,49.80,26,19,57.78,3.46,97.49,9.49,92.19,1.017
2,Seattle Kraken,82,4011.1667,27,49,6,24,60,0.366,3537,3455,50.59,2634,2622,50.11,1883,1879,50.05,152,188,44.71,144.55,154.18,48.39,1680,1692,49.82,997,978,50.48,125,162,43.55,12.54,83.44,653,682,48.91,484,499,49.24,77,106,42.08,15.91,78.76,1027,1010,50.42,513,479,51.71,48,56,46.15,9.36,88.31,1676,1572,51.60,827,805,50.67,27,26,50.94,3.26,96.77,8.07,89.99,0.981
3,Vegas Golden Knights,82,4076.3333,43,31,8,39,94,0.573,4147,3742,52.57,3100,2664,53.78,2279,1955,53.83,184,166,52.57,187.02,168.36,52.63,2021,1905,51.48,1205,1088,52.55,152,139,52.23,12.61,87.22,847,779,52.09,615,558,52.43,102,94,52.04,16.59,83.15,1174,1126,51.04,590,530,52.68,50,45,52.63,8.47,91.51,1851,1579,53.97,950,758,55.62,31,26,54.39,3.26,96.57,8.07,91.51,0.996
4,Montreal Canadiens,82,3928.1333,22,49,11,19,55,0.335,3422,3871,46.92,2532,2927,46.38,1902,2167,46.74,154,195,44.13,155.42,183.32,45.88,1680,2037,45.20,974,1216,44.47,129,168,43.43,13.24,86.18,702,842,45.47,495,613,44.68,72,122,37.11,14.55,80.10,978,1195,45.01,479,603,44.27,57,46,55.34,11.90,92.37,1505,1592,48.60,826,847,49.37,25,26,49.02,3.03,96.93,8.10,91.00,0.991


In [94]:
NHL_5v5_2122['W'] = NHL_5v5_2122['W'].mask(NHL_5v5_2122['Team'] == 'Tampa Bay Lightning', 51)
NHL_5v5_2122['L'] = NHL_5v5_2122['L'].mask(NHL_5v5_2122['Team'] == 'Tampa Bay Lightning', 23)
NHL_5v5_2122['Points'] = NHL_5v5_2122['Points'].mask(NHL_5v5_2122['Team'] == 'Tampa Bay Lightning', 110)
NHL_5v5_2122['GP'] = NHL_5v5_2122['GP'].mask(NHL_5v5_2122['Team'] == 'Tampa Bay Lightning', 82)
NHL_5v5_2122['Point %'] = NHL_5v5_2122['Point %'].mask(NHL_5v5_2122['Team'] == 'Tampa Bay Lightning', 0.671)
NHL_5v5_2122['ROW'] = NHL_5v5_2122['ROW'].mask(NHL_5v5_2122['Team'] == 'Tampa Bay Lightning', 49)

In [95]:
NHL_5v5_2122['L'] = NHL_5v5_2122['L'].mask(NHL_5v5_2122['Team'] == 'Toronto Maple Leafs', 21)
NHL_5v5_2122['GP'] = NHL_5v5_2122['GP'].mask(NHL_5v5_2122['Team'] == 'Toronto Maple Leafs', 82)
NHL_5v5_2122['Point %'] = NHL_5v5_2122['Point %'].mask(NHL_5v5_2122['Team'] == 'Toronto Maple Leafs', 0.701)

In [98]:
NHL_PP_2122['W'] = NHL_PP_2122['W'].mask(NHL_PP_2122['Team'] == 'Tampa Bay Lightning', 51)
NHL_PP_2122['L'] = NHL_PP_2122['L'].mask(NHL_PP_2122['Team'] == 'Tampa Bay Lightning', 23)
NHL_PP_2122['Points'] = NHL_PP_2122['Points'].mask(NHL_PP_2122['Team'] == 'Tampa Bay Lightning', 110)
NHL_PP_2122['GP'] = NHL_PP_2122['GP'].mask(NHL_PP_2122['Team'] == 'Tampa Bay Lightning', 82)
NHL_PP_2122['Point %'] = NHL_PP_2122['Point %'].mask(NHL_PP_2122['Team'] == 'Tampa Bay Lightning', 0.671)
NHL_PP_2122['ROW'] = NHL_PP_2122['ROW'].mask(NHL_PP_2122['Team'] == 'Tampa Bay Lightning', 49)

In [99]:
NHL_PP_2122['L'] = NHL_PP_2122['L'].mask(NHL_PP_2122['Team'] == 'Toronto Maple Leafs', 21)
NHL_PP_2122['GP'] = NHL_PP_2122['GP'].mask(NHL_PP_2122['Team'] == 'Toronto Maple Leafs', 82)
NHL_PP_2122['Point %'] = NHL_PP_2122['Point %'].mask(NHL_PP_2122['Team'] == 'Toronto Maple Leafs', 0.701)

In [100]:
filtered_df = NHL_PP_2122[NHL_PP_2122['Team'].isin(['Toronto Maple Leafs', 'Tampa Bay Lightning'])]
filtered_df

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCSF,SCSA,SCSF%,SCGF,SCGA,SCGF%,SCSH%,SCSV%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO
26,Tampa Bay Lightning,82,421.0167,51,23,8,49,110,0.671,683,112,85.91,559,97,85.21,411,74,84.74,62,5,92.54,54.78,7.62,87.79,394,66,85.65,250,45,84.75,50,4,92.59,20.00,91.11,170,31,84.58,124,26,82.67,29,3,90.63,23.39,88.46,224,35,86.49,126,19,86.90,21,1,95.45,16.67,94.74,389,25,93.96,159,10,94.08,11,0,100.00,6.92,100.00,15.09,93.24,1.083
27,Toronto Maple Leafs,82,356.6167,54,21,7,51,115,0.701,670,98,87.24,538,87,86.08,384,75,83.66,63,4,94.03,54.54,6.73,89.02,413,48,89.59,247,38,86.67,54,3,94.74,21.86,92.11,162,24,87.10,116,22,84.06,28,2,93.33,24.14,90.91,251,24,91.27,131,16,89.12,26,1,96.30,19.85,93.75,265,26,91.07,136,16,89.47,8,1,88.89,5.88,93.75,16.41,94.67,1.111


In [101]:
NHL_PP_2122.head()

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCSF,SCSA,SCSF%,SCGF,SCGA,SCGF%,SCSH%,SCSV%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO
0,Anaheim Ducks,82,371.1833,31,37,14,27,76,0.463,573,99,85.27,405,90,81.82,289,74,79.61,49,4,92.45,35.09,5.65,86.14,282,50,84.94,158,39,80.20,35,4,89.74,22.15,89.74,99,23,81.15,72,18,80.00,23,3,88.46,31.94,83.33,183,27,87.14,86,21,80.37,12,1,92.31,13.95,95.24,289,16,94.75,129,9,93.48,13,0,100.00,10.08,100.00,16.96,94.59,1.115
1,Arizona Coyotes,82,349.8167,25,50,7,24,57,0.348,445,97,82.10,346,87,79.91,255,69,78.70,28,4,87.50,32.81,6.71,83.02,228,43,84.13,149,31,82.78,21,4,84.00,14.09,87.10,103,19,84.43,73,16,82.02,10,1,90.91,13.70,93.75,125,24,83.89,76,15,83.52,11,3,78.57,14.47,80.00,216,23,90.38,106,10,91.38,7,0,100.00,6.60,100.00,10.98,94.20,1.052
2,Boston Bruins,82,386.4667,51,26,5,49,107,0.652,708,127,84.79,552,116,82.63,393,96,80.37,50,6,89.29,49.81,8.18,85.90,390,67,85.34,241,46,83.97,41,6,87.23,17.01,86.96,140,32,81.40,107,29,78.68,17,5,77.27,15.89,82.76,250,35,87.72,134,17,88.74,24,1,96.00,17.91,94.12,320,24,93.02,152,15,91.02,9,0,100.00,5.92,100.00,12.72,93.75,1.065
3,Buffalo Sabres,82,383.4000,32,39,11,29,75,0.457,555,101,84.60,439,93,82.52,326,79,80.49,47,6,88.68,35.90,7.19,83.31,260,48,84.42,161,39,80.50,30,5,85.71,18.63,87.18,95,29,76.61,68,26,72.34,13,5,72.22,19.12,80.77,165,19,89.67,93,13,87.74,17,0,100.00,18.28,100.00,295,24,92.48,165,16,91.16,17,1,94.44,10.30,93.75,14.42,92.41,1.068
4,Carolina Hurricanes,82,368.7000,54,20,8,53,116,0.707,632,96,86.81,513,87,85.50,364,70,83.87,51,3,94.44,48.59,6.11,88.84,350,49,87.72,208,30,87.39,44,3,93.62,21.15,90.00,144,26,84.71,95,17,84.82,26,3,89.66,27.37,82.35,206,23,89.96,113,13,89.68,18,0,100.00,15.93,100.00,286,14,95.33,153,10,93.87,7,0,100.00,4.58,100.00,14.01,95.71,1.097


In [102]:
NHL_PK_2122['W'] = NHL_PK_2122['W'].mask(NHL_PK_2122['Team'] == 'Tampa Bay Lightning', 51)
NHL_PK_2122['L'] = NHL_PK_2122['L'].mask(NHL_PK_2122['Team'] == 'Tampa Bay Lightning', 23)
NHL_PK_2122['Points'] = NHL_PK_2122['Points'].mask(NHL_PK_2122['Team'] == 'Tampa Bay Lightning', 110)
NHL_PK_2122['GP'] = NHL_PK_2122['GP'].mask(NHL_PK_2122['Team'] == 'Tampa Bay Lightning', 82)
NHL_PK_2122['Point %'] = NHL_PK_2122['Point %'].mask(NHL_PK_2122['Team'] == 'Tampa Bay Lightning', 0.671)
NHL_PK_2122['ROW'] = NHL_PK_2122['ROW'].mask(NHL_PK_2122['Team'] == 'Tampa Bay Lightning', 49)

In [103]:
NHL_PK_2122['L'] = NHL_PK_2122['L'].mask(NHL_PK_2122['Team'] == 'Toronto Maple Leafs', 21)
NHL_PK_2122['GP'] = NHL_PK_2122['GP'].mask(NHL_PK_2122['Team'] == 'Toronto Maple Leafs', 82)
NHL_PK_2122['Point %'] = NHL_PK_2122['Point %'].mask(NHL_PK_2122['Team'] == 'Toronto Maple Leafs', 0.701)

In [104]:
filtered_df = NHL_PK_2122[NHL_PP_2122['Team'].isin(['Toronto Maple Leafs', 'Tampa Bay Lightning'])]
filtered_df

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCSF,SCSA,SCSF%,SCGF,SCGA,SCGF%,SCSH%,SCSV%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO
26,Tampa Bay Lightning,82,425.5167,51,23,8,49,110,0.671,96,652,12.83,80,484,14.18,65,358,15.37,7,49,12.50,7.1,45.35,13.53,55,380,12.64,38,219,14.79,6,41,12.77,15.79,81.28,29,147,16.48,25,105,19.23,5,26,16.13,20.00,75.24,26,233,10.04,13,114,10.24,1,15,6.25,7.69,86.84,26,353,6.86,10,139,6.71,1,8,11.11,10.0,94.24,10.77,86.31,0.971
27,Toronto Maple Leafs,82,382.3500,54,21,7,51,115,0.701,147,506,22.51,131,410,24.21,101,312,24.46,13,42,23.64,9.8,38.11,20.45,81,263,23.55,57,176,24.46,13,36,26.53,22.81,79.55,37,107,25.69,28,79,26.17,10,17,37.04,35.71,78.48,44,156,22.00,29,97,23.02,3,19,13.64,10.34,80.41,39,253,13.36,21,134,13.55,0,6,0.00,0.0,95.52,12.87,86.54,0.994


Now all of the 21-22 df's look good.

The 22-23 and 23-24 seasons are good no manipulation needs to be done there. 

Now that we have our dataframes from NST, lets create a function to merge the primary dataframe obtained from the function with pp and pk dataframes. In other words, a function to merge 5v5, PP, and PK team season logs from NST. 

In [109]:
def merge_team_stats(primary_df, pp_df, pk_df):
    primary_df=primary_df.merge(pk_df, on='Team', how='left', suffixes = ('', '_pk'))
    primary_df=primary_df.merge(pp_df, on='Team', how='left', suffixes = ('', '_pp'))
    return primary_df

In [110]:
Final2122_df=merge_team_stats(NHL_5v5_2122,NHL_PP_2122,NHL_PK_2122)

In [111]:
Final2122_df.head()

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCSF,SCSA,SCSF%,SCGF,SCGA,SCGF%,SCSH%,SCSV%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,GP_pk,TOI_pk,W_pk,L_pk,OTL_pk,ROW_pk,Points_pk,Point %_pk,CF_pk,CA_pk,CF%_pk,FF_pk,FA_pk,FF%_pk,SF_pk,SA_pk,SF%_pk,GF_pk,GA_pk,GF%_pk,xGF_pk,xGA_pk,xGF%_pk,SCF_pk,SCA_pk,SCF%_pk,SCSF_pk,SCSA_pk,SCSF%_pk,SCGF_pk,SCGA_pk,SCGF%_pk,SCSH%_pk,SCSV%_pk,HDCF_pk,HDCA_pk,HDCF%_pk,HDSF_pk,HDSA_pk,HDSF%_pk,HDGF_pk,HDGA_pk,HDGF%_pk,HDSH%_pk,HDSV%_pk,MDCF_pk,MDCA_pk,MDCF%_pk,MDSF_pk,MDSA_pk,MDSF%_pk,MDGF_pk,MDGA_pk,MDGF%_pk,MDSH%_pk,MDSV%_pk,LDCF_pk,LDCA_pk,LDCF%_pk,LDSF_pk,LDSA_pk,LDSF%_pk,LDGF_pk,LDGA_pk,LDGF%_pk,LDSH%_pk,LDSV%_pk,SH%_pk,SV%_pk,PDO_pk,GP_pp,TOI_pp,W_pp,L_pp,OTL_pp,ROW_pp,Points_pp,Point %_pp,CF_pp,CA_pp,CF%_pp,FF_pp,FA_pp,FF%_pp,SF_pp,SA_pp,SF%_pp,GF_pp,GA_pp,GF%_pp,xGF_pp,xGA_pp,xGF%_pp,SCF_pp,SCA_pp,SCF%_pp,SCSF_pp,SCSA_pp,SCSF%_pp,SCGF_pp,SCGA_pp,SCGF%_pp,SCSH%_pp,SCSV%_pp,HDCF_pp,HDCA_pp,HDCF%_pp,HDSF_pp,HDSA_pp,HDSF%_pp,HDGF_pp,HDGA_pp,HDGF%_pp,HDSH%_pp,HDSV%_pp,MDCF_pp,MDCA_pp,MDCF%_pp,MDSF_pp,MDSA_pp,MDSF%_pp,MDGF_pp,MDGA_pp,MDGF%_pp,MDSH%_pp,MDSV%_pp,LDCF_pp,LDCA_pp,LDCF%_pp,LDSF_pp,LDSA_pp,LDSF%_pp,LDGF_pp,LDGA_pp,LDGF%_pp,LDSH%_pp,LDSV%_pp,SH%_pp,SV%_pp,PDO_pp
0,Pittsburgh Penguins,82,4030.0000,46,25,11,43,103,0.628,3905,3598,52.05,3031,2697,52.92,2257,2047,52.44,180,154,53.89,178.82,156.01,53.41,1968,1720,53.36,1247,1085,53.47,157,137,53.40,12.59,87.37,806,683,54.13,645,540,54.43,108,98,52.43,16.74,81.85,1162,1037,52.84,602,545,52.48,49,39,55.68,8.14,92.84,1742,1691,50.74,909,863,51.30,23,17,57.50,2.53,98.03,7.98,92.48,1.005,82,356.0500,46,25,11,43,103,0.628,86,568,13.15,77,437,14.98,67,305,18.01,3,33,8.33,4.74,40.03,10.59,38,281,11.91,28,155,15.30,3,25,10.71,10.71,83.87,20,97,17.09,16,66,19.51,1,16,5.88,6.25,75.76,18,184,8.91,12,89,11.88,2,9,18.18,16.67,89.89,18,294,5.77,10,149,6.29,0,8,0.00,0.00,94.63,4.48,89.18,0.937,82,399.4167,46,25,11,43,103,0.628,691,121,85.10,525,110,82.68,391,90,81.29,50,8,86.21,56.91,8.90,86.48,430,70,86.00,273,53,83.74,41,7,85.42,15.02,86.79,176,34,83.81,135,31,81.33,21,4,84.00,15.56,87.10,254,36,87.59,138,22,86.25,20,3,86.96,14.49,86.36,257,31,89.24,112,18,86.15,9,0,100.00,8.04,100.00,12.79,91.11,1.039
1,Tampa Bay Lightning,82,3870.4667,51,23,8,49,110,0.671,3550,3426,50.89,2736,2616,51.12,1929,1894,50.46,183,148,55.29,168.55,153.06,52.41,1928,1797,51.76,1119,1063,51.28,156,129,54.74,13.94,87.86,789,679,53.75,555,505,52.36,99,92,51.83,17.84,81.78,1139,1118,50.47,564,558,50.27,57,37,60.64,10.11,93.37,1771,1746,50.36,752,758,49.80,26,19,57.78,3.46,97.49,9.49,92.19,1.017,82,425.5167,51,23,8,49,110,0.671,96,652,12.83,80,484,14.18,65,358,15.37,7,49,12.50,7.10,45.35,13.53,55,380,12.64,38,219,14.79,6,41,12.77,15.79,81.28,29,147,16.48,25,105,19.23,5,26,16.13,20.00,75.24,26,233,10.04,13,114,10.24,1,15,6.25,7.69,86.84,26,353,6.86,10,139,6.71,1,8,11.11,10.00,94.24,10.77,86.31,0.971,82,421.0167,51,23,8,49,110,0.671,683,112,85.91,559,97,85.21,411,74,84.74,62,5,92.54,54.78,7.62,87.79,394,66,85.65,250,45,84.75,50,4,92.59,20.00,91.11,170,31,84.58,124,26,82.67,29,3,90.63,23.39,88.46,224,35,86.49,126,19,86.90,21,1,95.45,16.67,94.74,389,25,93.96,159,10,94.08,11,0,100.00,6.92,100.00,15.09,93.24,1.083
2,Seattle Kraken,82,4011.1667,27,49,6,24,60,0.366,3537,3455,50.59,2634,2622,50.11,1883,1879,50.05,152,188,44.71,144.55,154.18,48.39,1680,1692,49.82,997,978,50.48,125,162,43.55,12.54,83.44,653,682,48.91,484,499,49.24,77,106,42.08,15.91,78.76,1027,1010,50.42,513,479,51.71,48,56,46.15,9.36,88.31,1676,1572,51.60,827,805,50.67,27,26,50.94,3.26,96.77,8.07,89.99,0.981,82,359.6333,27,49,6,24,60,0.366,83,613,11.93,73,473,13.37,62,339,15.46,8,57,12.31,6.12,45.27,11.92,49,343,12.50,39,199,16.39,7,45,13.46,17.95,77.39,24,128,15.79,23,88,20.72,5,28,15.15,21.74,68.18,25,215,10.42,16,11

In [112]:
Final2223_df=merge_team_stats(NHL_5v5_2223,NHL_PP_2223,NHL_PK_2223)

In [114]:
Final2223_df.head()

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCSF,SCSA,SCSF%,SCGF,SCGA,SCGF%,SCSH%,SCSV%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,GP_pk,TOI_pk,W_pk,L_pk,OTL_pk,ROW_pk,Points_pk,Point %_pk,CF_pk,CA_pk,CF%_pk,FF_pk,FA_pk,FF%_pk,SF_pk,SA_pk,SF%_pk,GF_pk,GA_pk,GF%_pk,xGF_pk,xGA_pk,xGF%_pk,SCF_pk,SCA_pk,SCF%_pk,SCSF_pk,SCSA_pk,SCSF%_pk,SCGF_pk,SCGA_pk,SCGF%_pk,SCSH%_pk,SCSV%_pk,HDCF_pk,HDCA_pk,HDCF%_pk,HDSF_pk,HDSA_pk,HDSF%_pk,HDGF_pk,HDGA_pk,HDGF%_pk,HDSH%_pk,HDSV%_pk,MDCF_pk,MDCA_pk,MDCF%_pk,MDSF_pk,MDSA_pk,MDSF%_pk,MDGF_pk,MDGA_pk,MDGF%_pk,MDSH%_pk,MDSV%_pk,LDCF_pk,LDCA_pk,LDCF%_pk,LDSF_pk,LDSA_pk,LDSF%_pk,LDGF_pk,LDGA_pk,LDGF%_pk,LDSH%_pk,LDSV%_pk,SH%_pk,SV%_pk,PDO_pk,GP_pp,TOI_pp,W_pp,L_pp,OTL_pp,ROW_pp,Points_pp,Point %_pp,CF_pp,CA_pp,CF%_pp,FF_pp,FA_pp,FF%_pp,SF_pp,SA_pp,SF%_pp,GF_pp,GA_pp,GF%_pp,xGF_pp,xGA_pp,xGF%_pp,SCF_pp,SCA_pp,SCF%_pp,SCSF_pp,SCSA_pp,SCSF%_pp,SCGF_pp,SCGA_pp,SCGF%_pp,SCSH%_pp,SCSV%_pp,HDCF_pp,HDCA_pp,HDCF%_pp,HDSF_pp,HDSA_pp,HDSF%_pp,HDGF_pp,HDGA_pp,HDGF%_pp,HDSH%_pp,HDSV%_pp,MDCF_pp,MDCA_pp,MDCF%_pp,MDSF_pp,MDSA_pp,MDSF%_pp,MDGF_pp,MDGA_pp,MDGF%_pp,MDSH%_pp,MDSV%_pp,LDCF_pp,LDCA_pp,LDCF%_pp,LDSF_pp,LDSA_pp,LDSF%_pp,LDGF_pp,LDGA_pp,LDGF%_pp,LDSH%_pp,LDSV%_pp,SH%_pp,SV%_pp,PDO_pp
0,San Jose Sharks,82,3993.0000,22,44,16,21,60,0.366,3710,3923,48.60,2706,2890,48.36,1914,2059,48.18,168,220,43.30,184.11,184.26,49.98,1988,2031,49.47,1126,1159,49.28,143,199,41.81,12.70,82.83,858,809,51.47,580,559,50.92,91,113,44.61,15.69,79.79,1130,1222,48.04,546,600,47.64,52,86,37.68,9.52,85.67,1954,2126,47.89,704,803,46.72,25,21,54.35,3.55,97.38,8.78,89.32,0.981,82,423.8333,22,44,16,21,60,0.366,70,732,8.73,58,531,9.85,52,361,12.59,4,43,8.51,3.01,52.23,5.45,25,419,5.63,20,229,8.03,4,36,10.00,20.00,84.28,7,162,4.14,7,108,6.09,1,16,5.88,14.29,85.19,18,257,6.55,13,121,9.70,3,20,13.04,23.08,83.47,18,473,3.67,6,131,4.38,0,7,0.0,0.00,94.66,7.69,88.09,0.958,82,376.5333,22,44,16,21,60,0.366,596,108,84.66,450,91,83.18,334,73,82.06,41,13,75.93,47.39,9.29,83.62,347,49,87.63,211,39,84.40,31,12,72.09,14.69,69.23,146,29,83.43,105,26,80.15,19,9,67.86,18.10,65.38,201,20,90.95,106,13,89.08,12,3,80.00,11.32,76.92,389,40,90.68,121,14,89.63,10,1,90.91,8.26,92.86,12.28,82.19,0.945
1,Nashville Predators,82,3896.7000,42,32,8,36,92,0.561,3593,3985,47.41,2613,2952,46.95,1839,2086,46.85,146,153,48.83,168.00,186.61,47.38,1838,2037,47.43,1012,1166,46.46,122,131,48.22,12.06,88.77,756,819,48.00,513,570,47.37,83,80,50.92,16.18,85.96,1082,1218,47.04,499,596,45.57,39,51,43.33,7.82,91.44,1565,1715,47.71,746,812,47.88,24,22,52.17,3.22,97.29,7.94,92.67,1.006,82,457.1167,42,32,8,36,92,0.561,99,798,11.04,86,591,12.70,63,424,12.94,6,48,11.11,6.49,63.22,9.32,57,509,10.07,35,294,10.64,6,43,12.24,17.14,85.37,25,186,11.85,20,136,12.82,3,23,11.54,15.00,83.09,32,323,9.01,15,158,8.67,3,20,13.04,20.00,87.34,18,317,5.37,6,129,4.44,0,5,0.0,0.00,96.12,9.52,88.68,0.982,82,411.7000,42,32,8,36,92,0.561,693,124,84.82,523,108,82.88,360,93,79.47,44,6,88.00,49.26,9.61,83.67,380,70,84.44,200,53,79.05,30,6,83.33,15.00,88.68,164,35,82.41,108,29,78.83,22,3,88.00,20.37,89.66,216,35,86.06,92,24,79.31,8,3,72.73,8.70,87.50,306,22,93.29,157,12,92.90,14,0,100.00,8.92,100.00,12.22,93.55,1.058
2,Tampa Bay Lightning,82,3916.0833,46,30,6,43,98,0.598,3804,3590,51.45,2834,2700,51.21,1991,1992,49.99,175,162,51.93,180.07,170.06,51.43,1998,1847,51.96,1134,1086,51.08,148,134,52.48,13.05,87.66,853,756,53.01,590,557,51.44,104,77,57.46,17.63,86.18,1145,1091,51.21,544,529,50.70,44,57,43.56,8.09,89.22,2078,1914,52.05,788,822,48.94,26,28,48.15,3.30,96.59,8.79,91.87,1.007,82,418.9500,46,30,6,43,98,0.598,119,747,13.74,107,561,16.02,81,399,16.88,7,53,11.67,9.84,56.91,14.74,62,446,12.20,38,251,13.15,4,45,8.16,10.53,82.07,30,185,13.95,23,131,14.94,4,29,12.12,17.39,77.86,32,261,10.92,15,120,11.11,0,16,0.00

In [113]:
Final2324_df=merge_team_stats(NHL_5v5_2324,NHL_PP_2324,NHL_PK_2324)

Now have all team advanced stats in one data frame for each season. Now have to think about how we want to combine all three seasons into an ultimate dataframe to pass into the model to predict W, L, and Points for the 2024-2025 season. 

# Feature Engineering
